
# Financial Programming - IÉSEG, Lille - MBD 2019-2020
## Group Project - Financial Data

## Custom functions

In [1]:
#defining an explore function, summarizing the given dataset for data exploration
def explore(data,col):
    print(f"Summary of variables")
    print(data.describe())
    print()
    print(f"More information about the variables")
    print(data.info())
    print()
    print(f"The number of unique observations in the column selected is :")
    print(data[col].nunique())
#given the input of gender in the dataset, we corrected the +50 for female
def to_month_gender(birth_number):    
    s = str(birth_number)
    birth_month = int(s[2:4])
    if birth_month > 50:
        gender = "F"
        birth_month = birth_month - 50
    else:
        gender = 'M'
    return gender

## Packages, directory and data importation

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# import SQL in python using pandas
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [3]:
#/Users/spavot/Desktop/Big Data/Financial Programing/Assignment/Group Project_Customer Loan
#D:/User/Github/pythondatamart
directory = "/Users/spavot/Desktop/Big Data/Financial Programing/Assignment/Group Project_Customer Loan"
inputData = directory + "/Data"

In [4]:
#+"/Data"

# Each record describes static characteristics of an account
account = pd.read_csv(inputData+'/data_berka/account.asc', sep=';')

# Each record describes a credit card issued to an account
card = pd.read_csv(inputData+'/data_berka/card.asc', sep=';')

# Each record describes characteristics of a client
client = pd.read_csv(inputData+'/data_berka/client.asc', sep=';')

# Each record relates together a client with an account i.e. this relation describes the rights
# of clients to operate accounts
disp = pd.read_csv(inputData+'/data_berka/disp.asc', sep=';')

# Each record describes characteristics of a payment order
order = pd.read_csv(inputData+'/data_berka/order.asc', sep=';')

# Each record describes one transaction on an account
trans = pd.read_csv(inputData+'/data_berka/trans.asc', sep=';', low_memory=False)

# Each record describes a loan granted for a given account
loan = pd.read_csv(inputData+'/data_berka/loan.asc', sep=';')

# Each record describes demographic characteristics of a district.
district = pd.read_csv(inputData+'/data_berka/district.asc', sep=';')

## Data exploration and cleaning data

### Card

For the dataset Card, we want to have the type of card and the since time it was issued by taking the 1st of January 1999 as our reference date.

In [5]:
#change issuedate variabletype to datetime
card['issued'] = pd.to_datetime(card['issued'])

#we used the date of the project specified on the project description
card['date'] = "1999-01-01"
card['date'] = pd.to_datetime(card['date'])

#calculate how many days the card has been issued in days
card['Time since card issued'] = card['date'] - card['issued']

#We rename the columns to dodge duplicates during the merge
card = card.rename(columns = {'type':"Card type"})
card = card.rename(columns = {'issued':"Date card issued"})
card['Time since card issued'] = card['Time since card issued'].dt.days

#We nom drop our column use to calculate the time the cars has been issued because it will be useless in our final dataset
card = card.drop(columns = 'date')

#Finally, we create a column Has card which will be a flag column to be able to identify the client who have a card.
#Thus, it will gave the possibility to visualize data only on people who have card
card['Has card'] = 'Yes'

In [6]:
card.head()

,card_id,disp_id,Card type,Date card issued,Time since card issued,Has card
0,1005,9285,classic,1993-11-07,1881,Yes
1,104,588,classic,1994-01-19,1808,Yes
2,747,4915,classic,1994-02-05,1791,Yes
3,70,439,classic,1994-02-08,1788,Yes
4,577,3687,classic,1994-02-15,1781,Yes


### District

Many things have to be done in District data set. We will first rename the columns we want to use. Then, we're going to fix the missing value and we will create new variables such as trends of employment rate, percentage of entrepreneurs etc...

In [7]:
#renaming the district categories in readable, useful names
district.rename({'A1': 'district_id','A3':'Region', 'A2': 'district_name', 'A4': 'inhabitants',\
                 'A10': 'ratio_urban','A11': 'avg_salary','A12':'unempl_95',\
                 'A13':'unempl_96','A14':'entrepren','A15':'crime_95','A16':'crime_96'}, axis=1, inplace=True)

#now the creation of the new variables happens 
district['number_urban'] = round(district['inhabitants'] * (district['ratio_urban']/100))
#first replace the questionmarks to NA. 
district['unempl_95'].replace({'?': None},inplace =True)
district['crime_95'].replace({'?': None},inplace=True)

district['unempl_95']=pd.to_numeric(district['unempl_95'])
district['crime_95']=pd.to_numeric(district['crime_95'])

#now calculate new variables:people living on the countryside and the the two trends 
district['number_country']= district['inhabitants']-district['number_urban']
district['unemployment_trend']= "decreasing" 
district.loc[(district['unempl_95'] < district['unempl_96']), 'unemployment_trend']= 'increasing'
district.loc[(district['unempl_95'] == district['unempl_96']), 'unemployment_trend']= 'remains equal'

#convert number of entrepreneurs to percent
district['entrepren'] = ((district['entrepren']/10))

#for crime I chose to express it as crimes committed per inhabitant, but now the problem is should we do 95 or 96? 
#trend calculation is equal to unemployment
district['crime_per'] = (district['crime_96']/district['inhabitants'])
district['crime_rate']= "decreasing" 
district.loc[(district['crime_95'] < district['crime_96']), 'crime_rate']= 'increasing'
district.loc[(district['crime_95'] == district['crime_96']), 'crime_rate']= 'remains equal'

#We drop the columns we didn't want in our final dataset because we found them non relevant.
district = district.drop(columns =['A5','A6', 'A7', 'A8', 'A9'])


In [8]:
#To fix the missing value in crime_95 and unempl_95, we will create a flag column for those two columns and assign them a Yes
#if they have a rate and else a NO if the missing so we can visualize easily only on rate that we have.
#First, we replace NA with a special value and create the flag column for crime rate and assign all rows to YES
district['crime_95'] = district['crime_95'].fillna(99999999)
district['has crime rate'] = 'YES'

#Now, thanks to the value we've created we're going to switch to NO all values were the missing were replace by 99999999.
#Just after, we will replace all value 99999999 by 0 and so we fixed the missings values.
district.loc[district['crime_95'] == 99999999, 'has crime rate'] = 'NO'
district.loc[district['crime_95'] == 99999999, 'crime_95'] = 0

#We do the same operation as above for unempl_95:

district['unempl_95'] = district['unempl_95'].fillna(99999999)
district['has unempl rate'] = 'YES'
district.loc[district['unempl_95'] == 99999999, 'has crime rate'] = 'NO'
district.loc[district['unempl_95'] == 99999999, 'unempl_95'] = 0


In [9]:
district.head()

,district_id,district_name,Region,inhabitants,ratio_urban,avg_salary,unempl_95,unempl_96,entrepren,crime_95,crime_96,number_urban,number_country,unemployment_trend,crime_per,crime_rate,has crime rate,has unempl rate
0,1,Hl.m. Praha,Prague,1204953,100.0,12541,0.29,0.43,16.7,85677.0,99107,1204953.0,0.0,increasing,0.082250,increasing,YES,YES
1,2,Benesov,central Bohemia,88884,46.7,8507,1.67,1.85,13.2,2159.0,2674,41509.0,47375.0,increasing,0.030084,increasing,YES,YES
2,3,Beroun,central Bohemia,75232,41.7,8980,1.95,2.21,11.1,2824.0,2813,31372.0,43860.0,increasing,0.037391,decreasing,YES,YES
3,4,Kladno,central Bohemia,149893,67.4,9753,4.64,5.05,10.9,5244.0,5892,101028.0,48865.0,increasing,0.039308,increasing,YES,YES
4,5,Kolin,central Bohemia,95616,51.4,9307,3.85,4.43,11.8,2616.0,3040,49147.0,46469.0,increasing,0.031794,increasing,YES,YES


### Loan

For Loan dataset, we want to create new variables such as the remaining time of the loan, is the loan finished or not, how much time remains to fulfill the loan etc...

In [10]:
#Such as District or Card, we create a flag column to identify all people who contracted a loan.
loan['contracted a loan'] = "Yes"

#We rename the date column to date loan issued in order to avoid duplicates names at the end.
loan['date loan issued'] = loan['date']
loan = loan.drop(columns = 'date')

#Date loan issued is not a date yet, we're going to convert it to a date format to be able to execute data calculation:
loan['date loan issued'] = pd.to_datetime(loan['date loan issued'], format = '%y%m%d', yearfirst=True)

#We then create a date of the first january to calculate the time since the loan was done
loan['today'] = "1999-01-01"

#We transform it into date format
loan['today'] = pd.to_datetime(loan['today'])

#We create a new column which will be the difference in days between the date Today (that we defined) and the date the
#loan was issued
loan['month_since_issued'] = (loan['today'] - loan['date loan issued'])

#We convert this days difference from date format into integer
loan['month_since_issued'] = loan['month_since_issued'].dt.days

#We then divide by 30.41 and round it to have the number of month since it was issued
loan['month_since_issued'] = round(loan['month_since_issued']/30.41)

#We now create a new column that we will use to define it the loan is finished or not.
#First, we will compute the difference between the loan duration and the number of month since it was issued.
loan['month duration'] = loan['duration'] - loan['month_since_issued']

#So now, month duration is above 0 it means that it remains months for the loan, else if it's under 0 it means that the 
#loan duration is finished and so it's closed.
loan.loc[loan['month duration'] > 0, 'Loan_finished'] = "No"
loan.loc[loan['month duration'] <= 0, 'Loan_finished'] = "Yes"

#Now we will create a column which compute the number of months remaining before the loan finished
#We first assign this new column to zero
loan['Months remaining'] = 0

#Then, we're going to compute for the month duration we calculated before if it's more than 0 and if it's the case
#we assign this value to Months remaining column.
loan.loc[loan['month duration'] > 0, 'Months remaining'] = loan['month duration']

#We create now one last column computing the remaining amount to be payed:
#It is basically the result of the multiplication of months remaining by monthly payments
loan['Amount loan remaining'] = loan['payments']*loan['Months remaining']

#Now, we drop all the columns we used and that we don't want in our final dataset
loan = loan.drop(columns = 'month_since_issued')
loan = loan.drop(columns = 'month duration')
loan = loan.drop(columns = 'today')
loan = loan.drop(columns = 'loan_id')

In [11]:
loan.head()

,account_id,amount,duration,payments,status,contracted a loan,date loan issued,Loan_finished,Months remaining,Amount loan remaining
0,1787,96396,12,8033.0,B,Yes,1993-07-05,Yes,0.0,0.0
1,1801,165960,36,4610.0,A,Yes,1993-07-11,Yes,0.0,0.0
2,9188,127080,60,2118.0,A,Yes,1993-07-28,Yes,0.0,0.0
3,1843,105804,36,2939.0,A,Yes,1993-08-03,Yes,0.0,0.0
4,11013,274740,60,4579.0,A,Yes,1993-09-06,Yes,0.0,0.0


### Order

In order we will first rename value into english to be understandable for all, then we will only compute the number of order and the total amount paid by leasing (others values like insurrance etc.. are computed in transaction so we don't do twice to avoid duplicates

In [12]:
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


In [13]:
#First, we will create the total order per type
#To do this, we need first to fix missing values of k_symbol
order.loc[order['k_symbol'] == ' ', 'k_symbol'] = "OTHER"

#We will also create a column leasing and assigning it to 0. Then, we will assign to it the value of amount if the name of
#k_symbol is equal to LEASING. We only do leasing because other orders are already in transaction data set.
order['Leasing'] = 0
order.loc[order['k_symbol'] == 'LEASING' , 'Leasing'] = order['amount']

#Then, we do the sum per account ID using the group by method.
order = order.groupby(['account_id']).sum()

#We drop the columns we don't want to use
order = order.drop(columns = ['order_id','account_to'])

#And we rename amount column to avoid duplicates
order = order.rename(columns = {'amount' :'Total_order'})

#We create account it equal to index and reset it.
order['account_id'] = order.index
order = order.reset_index(drop=True)

In [14]:
order.head()

,Total_order,Leasing,account_id
0,2452.0,0.0,1
1,10638.7,0.0,2
2,5001.0,0.0,3
3,3363.0,0.0,4
4,2668.0,0.0,5


### Disp

In disp, we want to know if the account is shared and if the client is owner or the disponent.

In [15]:
#times one to convert from boolean to integer
shared=disp

#We create Is_Shared? column which is going to be equal to 1 if in owner the type is disponent.
shared['Is_shared?'] = (shared['type'] == 'DISPONENT')*1

#We drop the columns we don't want
shared=shared.drop(columns = ['disp_id','client_id','type'])

#Finally we group by the account_id using sum argument.
shared=shared.groupby(['account_id']).sum()

#Finally, we rename the column to avoid duplicates
disp = disp.rename(columns = {"type":"Owner / Disp"})

In [16]:
disp.head()

,disp_id,client_id,account_id,Owner / Disp,Is_shared?
0,1,1,1,OWNER,0
1,2,2,2,OWNER,0
2,3,3,2,DISPONENT,1
3,4,4,3,OWNER,0
4,5,5,3,DISPONENT,1


### Account

We're going to create variable for recency in years and also rename frequency variable into english

In [17]:
#using the earlier defined explore function for data exploration 
explore(account,"date")

#calculation of recency, in years.
#First, we create a new variable which is the difference between date and 1999
account["Recency years"]=1999-(1900+round(account["date"]/10000,0))

#We convert the date of account opened to a date format
account['date'] = pd.to_datetime(account['date'], format = '%y%m%d', yearfirst=True)

#Now we rename date to avoid duplicates
account = account.rename(columns = {"date": "Date account opened"})

#Finally, we rename variable into frequency:
account.loc[account['frequency'] == 'POPLATEK MESICNE', 'frequency'] = "Montlhy"
account.loc[account['frequency'] == 'POPLATEK TYDNE', 'frequency'] = "Weekly"
account.loc[account['frequency'] == 'POPLATEK PO OBRATU', 'frequency'] = "Immediatly"

Summary of variables
         account_id  district_id           date
count   4500.000000  4500.000000    4500.000000
mean    2786.067556    37.310444  951654.608667
std     2313.811984    25.177217   14842.188377
min        1.000000     1.000000  930101.000000
25%     1182.750000    13.000000  931227.000000
50%     2368.000000    38.000000  960102.000000
75%     3552.250000    60.000000  961101.000000
max    11382.000000    77.000000  971229.000000

More information about the variables
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
account_id     4500 non-null int64
district_id    4500 non-null int64
frequency      4500 non-null object
date           4500 non-null int64
dtypes: int64(3), object(1)
memory usage: 140.7+ KB
None

The number of unique observations in the column selected is :
1535


In [18]:
account.head()

,account_id,district_id,frequency,Date account opened,Recency years
0,576,55,Montlhy,1993-01-01,6.0
1,3818,74,Montlhy,1993-01-01,6.0
2,704,55,Montlhy,1993-01-01,6.0
3,2378,16,Montlhy,1993-01-01,6.0
4,2632,24,Montlhy,1993-01-02,6.0


### Client

We want the age and the gender of the client

In [19]:
#using the explore function once more
explore(client,"client_id")
#calculation of the clients age 
client['age'] = 1999-client['birth_number'].transform(lambda bn: int('19' + str(bn)[:2]))
#transforming the birth_number into gender using the custom function
client['gender'] = client['birth_number'].apply(to_month_gender)
#birth_number variable not longer needed
client=client.drop(columns=['birth_number'])

Summary of variables
          client_id   birth_number  district_id
count   5369.000000    5369.000000  5369.000000
mean    3359.011920  535114.970013    37.310114
std     2832.911984  172895.618429    25.043690
min        1.000000  110820.000000     1.000000
25%     1418.000000  406009.000000    14.000000
50%     2839.000000  540829.000000    38.000000
75%     4257.000000  681013.000000    60.000000
max    13998.000000  875927.000000    77.000000

More information about the variables
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 3 columns):
client_id       5369 non-null int64
birth_number    5369 non-null int64
district_id     5369 non-null int64
dtypes: int64(3)
memory usage: 125.9 KB
None

The number of unique observations in the column selected is :
5369


In [20]:
client.head()

,client_id,district_id,age,gender
0,1,18,29,F
1,2,1,54,M
2,3,1,59,F
3,4,5,43,M
4,5,5,39,F


### Transactions

For transactions, we want to transform all the transactions into summarize per accoutn so we can then merge it in our final data mart.

In [21]:
trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [22]:
trans.nunique()

trans_id      1056320
account_id       4500
date             2191
type                3
operation           5
amount          40400
balance        542739
k_symbol            8
bank               13
account          7665
dtype: int64

In [23]:
#First, we will replace missing value in k_symbol by OTHER
trans.loc[trans['k_symbol'] == ' ', 'k_symbol'] = "OTHER"
trans['k_symbol'] = trans['k_symbol'].fillna("OTHER")

#We do the same for the bank and operation column
trans['bank'] = trans['bank'].fillna("OTHER")
trans['operation'] = trans['operation'].fillna("OTHER")

In [24]:
#We will rename then the k_symbol variables into english
trans.loc[trans['k_symbol'] == 'POJISTNE', 'k_symbol'] = "Insurrance payment"
trans.loc[trans['k_symbol'] == 'SLUZBY', 'k_symbol'] = "Statement payment"
trans.loc[trans['k_symbol'] == 'UROK', 'k_symbol'] = "Interest credited"
trans.loc[trans['k_symbol'] == 'SANKC. UROK', 'k_symbol'] = "Sanction interest negative"
trans.loc[trans['k_symbol'] == 'SIPO', 'k_symbol'] = "Household"
trans.loc[trans['k_symbol'] == 'DUCHOD', 'k_symbol'] = "Age pension"
trans.loc[trans['k_symbol'] == 'UVER', 'k_symbol'] = "Loan payment"

In [25]:
#Now, we're doing the same but for the operation variables
trans.loc[trans['operation'] == 'VYBER', 'operation'] = "Cash withdraw"
trans.loc[trans['operation'] == 'PREVOD NA UCET', 'operation'] = "Money transfer to other bank"
trans.loc[trans['operation'] == 'PREVOD Z UCTU', 'operation'] = "Recovering other bank"
trans.loc[trans['operation'] == 'VKLAD', 'operation'] = "Cash deposit"
trans.loc[trans['operation'] == 'VYBER KARTOU', 'operation'] = "Debit card"

#And for type variables
trans.loc[trans['type'] == 'VYDAJ', 'type'] = "Debit"
trans.loc[trans['type'] == 'PRIJEM', 'type'] = "Credit"

In [26]:
#We now are going to compute a new column which will be used later to be the sum of debit and another one for credit
trans['Credit'] = 0
trans['Debit'] = 0

#After initializing them to 0, if it's credit then the amount of the transaction will be assign to Credit column and vice versa
trans.loc[trans['type'] == 'Credit' , 'Credit'] = trans['amount']
trans.loc[trans['type'] == 'Debit' , 'Debit'] = trans['amount']

In [27]:
#Now, we're going to do the same for each type of operation.
#First, we initialize them as column full of 0
trans['Cash deposit'] = 0
trans['Cash withdraw'] = 0
trans['Money transfer to other bank'] = 0
trans['Recovering other bank'] = 0
trans['Debit card'] = 0
trans['Other operation'] = 0

In [28]:
#Now for each type of operation, we're going to assign the value in the column we created for this:
trans.loc[trans['operation'] == 'Cash deposit' , 'Cash deposit'] = trans['amount']
trans.loc[trans['operation'] == 'Cash withdraw' , 'Cash withdraw'] = trans['amount']
trans.loc[trans['operation'] == 'Money transfer to other bank' , 'Money transfer to other bank'] = trans['amount']
trans.loc[trans['operation'] == 'Recovering other bank' , 'Recovering other bank'] = trans['amount']
trans.loc[trans['operation'] == 'Debit card' , 'Debit card'] = trans['amount']
trans.loc[trans['operation'] == 'OTHER' , 'Other operation'] = trans['amount']

In [29]:
#We do the same with the k_symbol variables, intialize them as 0.
trans['Insurrance payment'] = 0
trans['Statement payment'] = 0
trans['Interest credited'] = 0
trans['Sanction interest negative'] = 0
trans['Household'] = 0
trans['Age pension'] = 0
trans['Loan payment'] = 0
trans['Other transaction'] = 0

In [30]:
#Now for each type of k_symbol, the amount is assigned to their respective column
trans.loc[trans['k_symbol'] == 'Insurrance payment' , 'Insurrance payment'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Statement payment' , 'Statement payment'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Interest credited' , 'Interest credited'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Sanction interest negative' , 'Sanction interest negative'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Household' , 'Household'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Age pension' , 'Age pension'] = trans['amount']
trans.loc[trans['k_symbol'] == 'Loan payment' , 'Loan payment'] = trans['amount']
trans.loc[trans['k_symbol'] == 'OTHER' , 'Other transaction'] = trans['amount']

In [31]:
#We rename the column type to avoid duplicates
trans = trans.rename(columns = {"type": "Credit / Debit"})

In [32]:
trans.head()

,trans_id,account_id,date,Credit / Debit,operation,amount,balance,k_symbol,bank,account,...,Debit card,Other operation,Insurrance payment,Statement payment,Interest credited,Sanction interest negative,Household,Age pension,Loan payment,Other transaction
0,695247,2378,930101,Credit,Cash deposit,700.0,700.0,OTHER,OTHER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0
1,171812,576,930101,Credit,Cash deposit,900.0,900.0,OTHER,OTHER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,900.0
2,207264,704,930101,Credit,Cash deposit,1000.0,1000.0,OTHER,OTHER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0
3,1117247,3818,930101,Credit,Cash deposit,600.0,600.0,OTHER,OTHER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0
4,579373,1972,930102,Credit,Cash deposit,400.0,400.0,OTHER,OTHER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0


In [33]:
#We initiate a column count to 1 to be used to do count the number of transactions.
trans['count'] = 1 

In [34]:
#We now drop the columns that we don't want anymore
trans =  trans.drop(columns ='trans_id')
trans =  trans.drop(columns ='date')
trans = trans.drop(columns = 'amount')
trans = trans.drop(columns = 'balance')
trans = trans.drop(columns = 'account')

#We're goruping by account id using sum to have the sum per account. All nominal values are going to be loosed this is why
#we create the new columns so we don't loose the nominal values as they're now columns
trans = trans.groupby('account_id').sum()

#Finally we create the column solde which is the result of credit minus debit.
trans['solde'] = trans['Credit'] - trans['Debit']
trans.head()

,Credit,Debit,Cash deposit,Cash withdraw,Money transfer to other bank,Recovering other bank,Debit card,Other operation,Insurrance payment,Statement payment,Interest credited,Sanction interest negative,Household,Age pension,Loan payment,Other transaction,count,solde
account_id,,,,,,,,,,,,,,,,,,
1,194320.5,180854.0,25700.0,82774.0,98080.0,165555.0,0.0,3065.5,0.0,584.0,3065.5,0.0,98080.0,0.0,0.0,273445.0,239,13466.5
2,1597053.5,1336983.8,48400.0,1001191.0,553234.8,1537936.0,0.0,10717.5,0.0,949.0,10717.5,0.0,472290.0,0.0,80944.8,2586578.0,478,260069.7
3,173059.0,121962.8,170989.0,56949.8,65013.0,0.0,0.0,2070.0,46007.0,189.8,2070.0,0.0,14755.0,0.0,0.0,232000.0,117,51096.2
4,192346.0,158625.4,800.0,59813.4,98812.0,188802.0,0.0,2744.0,0.0,423.4,2744.0,0.0,60262.0,188802.0,0.0,98740.0,186,33720.6
5,97485.0,69396.4,600.0,32044.4,37352.0,95323.0,0.0,1562.0,0.0,204.4,1562.0,0.0,37352.0,95323.0,0.0,32440.0,84,28088.6


In [35]:
trans.describe()

,Credit,Debit,Cash deposit,Cash withdraw,Money transfer to other bank,Recovering other bank,Debit card,Other operation,Insurrance payment,Statement payment,Interest credited,Sanction interest negative,Household,Age pension,Loan payment,Other transaction,count,solde
count,4.500000e+03,4.500000e+03,4.500000e+03,4.500000e+03,4.500000e+03,4.500000e+03,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4.500000e+03,4500.000000,4500.000000
mean,7.172161e+05,6.270483e+05,5.374493e+05,5.198918e+05,1.494751e+05,1.736622e+05,4037.866667,6104.597667,5372.265111,596.981822,6104.597667,8.369622,110779.203556,37216.026222,12278.289622,1.218265e+06,234.737778,90167.788156
std,6.702457e+05,5.845288e+05,6.467904e+05,5.957891e+05,1.585267e+05,4.535806e+05,13094.403138,4272.782647,30238.204556,595.226671,4272.782647,98.592272,134595.169719,94904.473226,39466.681504,1.288805e+06,126.849443,112954.895403
min,1.970000e+04,1.400000e+03,2.000000e+02,1.400000e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.189000e+04,9.000000,-25821.000000
25%,2.280506e+05,1.973261e+05,1.100000e+03,1.047545e+05,3.485125e+04,0.000000e+00,0.000000,2971.550000,0.000000,292.000000,2971.550000,0.000000,0.000000,0.000000,0.000000,2.365725e+05,133.000000,24889.700000
50%,4.666833e+05,4.133415e+05,3.346475e+05,2.844079e+05,1.062350e+05,0.000000e+00,0.000000,5055.650000,0.000000,467.200000,5055.650000,0.000000,68142.000000,0.000000,0.000000,7.773990e+05,208.000000,47054.700000
75%,1.005419e+06,8.832919e+05,8.178412e+05,7.156133e+05,2.111338e+05,1.433280e+05,0.000000,8168.275000,0.000000,817.600000,8168.275000,0.000000,161335.500000,0.000000,0.000000,1.754293e+06,330.000000,90898.250000
max,3.857257e+06,3.590676e+06,3.711558e+06,3.656226e+06,1.040262e+06,3.818430e+06,153900.000000,31208.900000,650208.000000,6900.000000,31208.900000,3083.700000,957700.000000,485380.000000,412850.000000,7.384295e+06,675.000000,832642.000000


## Merge

In [36]:
#merging the data, quite straightforward, see documentation for the reasoning
accountloan=pd.merge(account,loan,'left', on='account_id')
accountloanorder=pd.merge(accountloan,order,'left', on='account_id')
accountloanordertrans=pd.merge(accountloanorder,trans,'left', on='account_id')
accountdistrictloanordertrans=pd.merge(accountloanordertrans,district,'right',on='district_id')
dispcard=pd.merge(disp,card,'left', on='disp_id')
dispclientcard=pd.merge(dispcard,client,'right',on='client_id')
client_merge=pd.merge(dispclientcard,accountdistrictloanordertrans,'right',on='account_id')

In [37]:
#Some values are still missing in the merge because none of all clients have card of loan for example.
#If you remember, we created a flag variable for card and loan everything was assigned to yes.
#But now that we've merged everything, some client don't have any value here so because they don't have card,
#we're assigning them the value NO so we can flag them.
client_merge['contracted a loan'] = client_merge['contracted a loan'].fillna("NO")
client_merge['Has card'] = client_merge['Has card'].fillna("NO")

#Now everything has been flagged so we can replace all other missing value in other columns to 0.
client_merge['Total_order'] = client_merge['Total_order'].fillna(0)
client_merge['Leasing'] = client_merge['Leasing'].fillna(0)
client_merge = client_merge.fillna(0)

#We drop duplicates column and rename others one to have a clean and understanble datamart
client_merge = client_merge.drop(columns = 'district_id_x')
client_merge = client_merge.rename(columns = {'frequency':"Issuance type","recency years":"Seniority","amount":"Loan amount",\
                                              "duration":"Loan duration", "payments":"Loan payment by month", "status"\
                                              :"Loan status", "Month remaining":"Month loan remaining", "district_id_y"\
                                              :"district_id","count":"Number transactions", "solde":"Total balance"})


In [38]:
client_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 62 columns):
disp_id                         5369 non-null int64
client_id                       5369 non-null int64
account_id                      5369 non-null int64
Owner / Disp                    5369 non-null object
Is_shared?                      5369 non-null int32
card_id                         5369 non-null float64
Card type                       5369 non-null object
Date card issued                5369 non-null object
Time since card issued          5369 non-null float64
Has card                        5369 non-null object
age                             5369 non-null int64
gender                          5369 non-null object
district_id                     5369 non-null int64
Issuance type                   5369 non-null object
Date account opened             5369 non-null datetime64[ns]
Recency years                   5369 non-null float64
Loan amount                     5369 non

### The last step: we export the datamart into CSV to be able to produce visualization with Tableau

In [39]:
client_merge.to_excel(r'C:\Users\spavot\Documents\GitHub\FinancialProgramming_Group09/MPVPythonGroup.xlsx')